In [2]:
# !wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
# !unzip uncased_L-12_H-768_A-12.zip

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [4]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import numpy as np
import tensorflow as tf
import pandas as pd
from tqdm import tqdm

In [5]:
import json

with open('dataset.json') as fopen:
    data = json.load(fopen)
    
train_X = data['train_X']
train_Y = data['train_Y']
test_X = data['test_X']
test_Y = data['test_Y']

In [7]:
BERT_VOCAB = 'uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'uncased_L-12_H-768_A-12/bert_config.json'

tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=False)

In [8]:
EOS = 1

In [9]:
import youtokentome as yttm

bpe = yttm.BPE(model='bpe.model')

In [10]:
from unidecode import unidecode
from tqdm import tqdm

def get_inputs(x, y):
    input_ids, input_masks, segment_ids, ys = [], [], [], []
    for i in tqdm(range(len(x))):
        tokens_a = tokenizer.tokenize(unidecode(x[i]))
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        
        segment_id = [0] * len(tokens)
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_id)

        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        
        r = bpe.encode(y[i], output_type=yttm.OutputType.ID) + [EOS]
        if len([k for k in r if k == 0]):
            print(y[i], i)
            break
        
        ys.append(r)
        
    return input_ids, input_masks, segment_ids, ys

In [11]:
train_input_ids, train_input_masks, train_segment_ids, train_Y = get_inputs(train_X, train_Y)

100%|██████████| 200000/200000 [01:24<00:00, 2359.12it/s]


In [12]:
test_input_ids, test_input_masks, test_segment_ids, test_Y = get_inputs(test_X, test_Y)

100%|██████████| 5000/5000 [00:01<00:00, 2711.31it/s]


In [13]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
epoch = 20
batch_size = 16
warmup_proportion = 0.1
num_train_steps = len(train_input_ids)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [14]:
bert_config.vocab_size

30522

In [16]:
from collections import defaultdict

BASE_PARAMS = defaultdict(
    lambda: None,  # Set default value to None.

    # Input params
    default_batch_size=2048,  # Maximum number of tokens per batch of examples.
    default_batch_size_tpu=32768,
    max_length=256,  # Maximum number of tokens per example.

    # Model params
    initializer_gain=1.0,  # Used in trainable variable initialization.
    vocab_size=32000,  # Number of tokens defined in the vocabulary file.
    hidden_size=768,  # Model dimension in the hidden layers.
    num_hidden_layers=6,  # Number of layers in the encoder and decoder stacks.
    num_heads=8,  # Number of heads to use in multi-headed attention.
    filter_size=2048,  # Inner layer dimension in the feedforward network.

    # Dropout values (only used when training)
    layer_postprocess_dropout=0.1,
    attention_dropout=0.1,
    relu_dropout=0.1,

    # Training params
    label_smoothing=0.1,
    learning_rate=1.0,
    learning_rate_decay_rate=1.0,
    learning_rate_warmup_steps=16000,

    # Optimizer params
    optimizer_adam_beta1=0.9,
    optimizer_adam_beta2=0.997,
    optimizer_adam_epsilon=1e-09,

    # Default prediction params
    extra_decode_length=50,
    beam_size=4,
    alpha=0.6,  # used to calculate length normalization in beam search

    # TPU specific parameters
    use_tpu=False,
    static_batch=False,
    allow_ffn_pad=True,
)

In [19]:
from transformer import model_utils
from transformer import utils
from transformer.transformer import DecoderStack
from transformer import beam_search
from transformer import embedding_layer

class Model:
    def __init__(
        self,
        learning_rate = 2e-5,
        training = True,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=training,
            input_ids=self.X,
            input_mask=self.input_masks,
            token_type_ids=self.segment_ids,
            use_one_hot_embeddings=False)
        
        self.decoder_stack = DecoderStack(BASE_PARAMS, training)
        attention_bias = model_utils.get_padding_bias(self.X)
        
        output_layer = model.get_sequence_output()
        
        self.embedding_softmax_layer = embedding_layer.EmbeddingSharedWeights(
        BASE_PARAMS["vocab_size"], BASE_PARAMS["hidden_size"], 'gather')
        
        with tf.name_scope("decode"):
            decoder_inputs = self.embedding_softmax_layer(self.Y)
            with tf.name_scope("shift_targets"):
                decoder_inputs = tf.pad(
                    decoder_inputs, [[0, 0], [1, 0], [0, 0]])[:, :-1, :]
            with tf.name_scope("add_pos_encoding"):
                length = tf.shape(decoder_inputs)[1]
                decoder_inputs += model_utils.get_position_encoding(
                    length, BASE_PARAMS["hidden_size"])
            if training:
                decoder_inputs = tf.nn.dropout(
                    decoder_inputs, 1 - BASE_PARAMS["layer_postprocess_dropout"])
            decoder_self_attention_bias = model_utils.get_decoder_self_attention_bias(length)
            outputs = self.decoder_stack(
              decoder_inputs, output_layer, decoder_self_attention_bias,
              attention_bias)
            self.training_logits = self.embedding_softmax_layer.linear(outputs)
                
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        def _get_symbols_to_logits_fn(max_decode_length):
            timing_signal = model_utils.get_position_encoding(
                max_decode_length + 1, BASE_PARAMS["hidden_size"])
            decoder_self_attention_bias = model_utils.get_decoder_self_attention_bias(
                max_decode_length)
            def symbols_to_logits_fn(ids, i, cache):
                decoder_input = ids[:, -1:]
                decoder_input = self.embedding_softmax_layer(decoder_input)
                decoder_input += timing_signal[i:i + 1]
                self_attention_bias = decoder_self_attention_bias[:, :, i:i + 1, :i + 1]
                decoder_outputs = self.decoder_stack(
                  decoder_input, cache.get("encoder_outputs"), self_attention_bias,
                  cache.get("encoder_decoder_attention_bias"), cache)
                logits = self.embedding_softmax_layer.linear(decoder_outputs)
                logits = tf.squeeze(logits, axis=[1])
                return logits, cache
            return symbols_to_logits_fn
        
        batch_size = tf.shape(output_layer)[0]
        input_length = tf.shape(output_layer)[1]
        max_decode_length = input_length + BASE_PARAMS["extra_decode_length"]
        symbols_to_logits_fn = _get_symbols_to_logits_fn(max_decode_length)
        initial_ids = tf.zeros([batch_size], dtype=tf.int32)
        cache = {
        "layer_%d" % layer: {
            "k": tf.zeros([batch_size, 0, BASE_PARAMS["hidden_size"]]),
            "v": tf.zeros([batch_size, 0, BASE_PARAMS["hidden_size"]]),
        } for layer in range(BASE_PARAMS["num_hidden_layers"])}
        cache["encoder_outputs"] = output_layer
        cache["encoder_decoder_attention_bias"] = attention_bias
        
        decoded_ids, scores = beam_search.sequence_beam_search(
            symbols_to_logits_fn=symbols_to_logits_fn,
            initial_ids=initial_ids,
            initial_cache=cache,
            vocab_size=32000,
            beam_size=1,
            alpha=BASE_PARAMS["alpha"],
            max_decode_length=max_decode_length,
            eos_id=EOS)
        
        top_decoded_ids = decoded_ids[:, 0, 1:]
        self.fast_result = top_decoded_ids

In [20]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

sess.run(tf.global_variables_initializer())




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '



Instructions for updating:
Use `tf.cast` instead.






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [21]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if 'bert/' + name in name_to_variable:
            assignment_map[name] = name_to_variable['bert/' + name]
            initialized_variable_names[name] = 1
            initialized_variable_names[name + ':0'] = 1
        elif name in name_to_variable:
            assignment_map[name] = name_to_variable[name]
            initialized_variable_names[name] = 1
            initialized_variable_names[name + ':0'] = 1
        

    return (assignment_map, initialized_variable_names)

In [22]:
tvars = tf.trainable_variables()

checkpoint = BERT_INIT_CHKPNT
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                checkpoint)

In [23]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, checkpoint)

INFO:tensorflow:Restoring parameters from uncased_L-12_H-768_A-12/bert_model.ckpt


In [24]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [25]:
i = 0
index = min(i + batch_size, len(train_input_ids))
batch_x = train_input_ids[i: index]
batch_x = pad_sequences(batch_x, padding='post')
batch_mask = train_input_masks[i: index]
batch_mask = pad_sequences(batch_mask, padding='post')
batch_segment = train_segment_ids[i: index]
batch_segment = pad_sequences(batch_segment, padding='post')
batch_y = pad_sequences(train_Y[i: index], padding='post')
acc, cost, r = sess.run(
    [model.accuracy, model.cost, model.fast_result],
    feed_dict = {
        model.Y: batch_y,
        model.X: batch_x,
        model.input_masks: batch_mask,
        model.segment_ids: batch_segment
    },
)

In [27]:
from tqdm import tqdm
import time

for EPOCH in range(epoch):

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_mask = train_input_masks[i: index]
        batch_mask = pad_sequences(batch_mask, padding='post')
        batch_segment = train_segment_ids[i: index]
        batch_segment = pad_sequences(batch_segment, padding='post')
        batch_y = pad_sequences(train_Y[i: index], padding='post')
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.input_masks: batch_mask,
                model.segment_ids: batch_segment
            },
        )
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = pad_sequences(test_Y[i: index], padding='post')
        batch_mask = test_input_masks[i: index]
        batch_mask = pad_sequences(batch_mask, padding='post')
        batch_segment = test_segment_ids[i: index]
        batch_segment = pad_sequences(batch_segment, padding='post')
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.input_masks: batch_mask,
                model.segment_ids: batch_segment
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )

train minibatch loop:   0%|          | 0/12500 [00:00<?, ?it/s]

epoch: 0, training loss: 7.195237, training acc: 0.118680, valid loss: 5.926563, valid acc: 0.200815



train minibatch loop:   0%|          | 0/12500 [00:00<?, ?it/s]

epoch: 1, training loss: 5.066592, training acc: 0.272305, valid loss: 4.342011, valid acc: 0.340727



train minibatch loop:   0%|          | 0/12500 [00:00<?, ?it/s]

...

epoch: 17, training loss: 1.652442, training acc: 0.650099, valid loss: 2.350846, valid acc: 0.552391



train minibatch loop:   0%|          | 0/12500 [00:00<?, ?it/s]

epoch: 18, training loss: 1.652238, training acc: 0.649970, valid loss: 2.346917, valid acc: 0.551565



test minibatch loop: 100%|██████████| 313/313 [00:23<00:00, 13.45it/s, accuracy=0.565, cost=2.36]

epoch: 19, training loss: 1.651971, training acc: 0.650319, valid loss: 2.348038, valid acc: 0.553802



In [28]:
from tensor2tensor.utils import bleu_hook

In [29]:
results = []
for i in tqdm(range(0, len(test_X), batch_size)):
    index = min(i + batch_size, len(test_X))
    batch_x = test_input_ids[i: index]
    batch_x = pad_sequences(batch_x, padding='post')
    batch_y = pad_sequences(test_Y[i: index], padding='post')
    batch_mask = test_input_masks[i: index]
    batch_mask = pad_sequences(batch_mask, padding='post')
    batch_segment = test_segment_ids[i: index]
    batch_segment = pad_sequences(batch_segment, padding='post')
    feed = {
        model.X: batch_x,
        model.input_masks: batch_mask,
        model.segment_ids: batch_segment
    }
    p = sess.run(model.fast_result,feed_dict = feed)
    result = []
    for row in p:
        result.append([i for i in row if i > 1 and i not in [101, 102]])
    results.extend(result)

100%|██████████| 313/313 [08:32<00:00,  1.64s/it]


In [30]:
rights = []
for r in test_Y:
    rights.append([i for i in r if i > 1 and i not in [101, 102]])

In [31]:
bleu_hook.compute_bleu(reference_corpus = rights,
                       translation_corpus = results)

0.23225775